In [ ]:

import pandas as pd

reviews_path = "../data/raw/reviews_raw.csv"  # if notebook is in notebooks/
df_raw = pd.read_csv(reviews_path)

df_raw.head()
df_raw["bank_name"].value_counts(), df_raw["rating"].describe()

#  Imports and helpers
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, inspect
import psycopg2
sns.set(style="whitegrid")

def try_connect_db():
    # Try several environment variables commonly used for DB connections
    db_uri = os.getenv('DB_URI') or os.getenv('DATABASE_URL') or os.getenv('DB_URL')
    if not db_uri:
        # Try building from components if provided
        user = os.getenv('DB_USER', os.getenv('PGUSER', ''))
        password = os.getenv('DB_PASSWORD', os.getenv('PGPASSWORD', ''))
        host = os.getenv('DB_HOST', os.getenv('PGHOST', 'localhost'))
        port = os.getenv('DB_PORT', os.getenv('PGPORT', '5432'))
        db = os.getenv('DB_NAME', os.getenv('PGDATABASE', ''))
        if user and password and db:
            db_uri = f'postgresql+psycopg2://{user}:{'Henzi19$'}@{host}:{port}/{db}'
    if db_uri:
        try:
            engine = create_engine(db_uri, connect_args={})
            # quick test
            insp = inspect(engine)
            return engine, insp
        except Exception as e:
            print('DB connect failed:', e)
    return None, None

def load_reviews_from_db(engine, inspector):
    # Try common table names and return the first one that exists
    candidates = ['reviews', 'reviews_clean', 'reviews_raw', 'raw_reviews', 'processed_reviews']
    for table in candidates:
        try:
            if inspector.has_table(table):
                df = pd.read_sql_table(table, engine)
                print(f'Loaded {len(df)} rows from {table} (DB)')
                return df
        except Exception:
            pass
    # If none found, return None
    return None

ModuleNotFoundError: No module named 'matplotlib'

# Preprocessing EDA

This notebook performs exploratory data analysis on the scraped reviews dataset. It first attempts to load data from a PostgreSQL database (use environment variables to configure). If the database is not available, it falls back to local CSVs produced by the scraping/preprocessing steps (e.g., `data/processed/reviews_clean.csv` or `data/raw/reviews_raw.csv`).

In [ ]:
# Attempt to load from DB, otherwise from local files
engine, insp = try_connect_db()
df = None
if engine is not None and insp is not None:
    try:
        df = load_reviews_from_db(engine, insp)
    except Exception as e:
        print('Error loading from DB:', e)

# Fallback candidates for CSV files
csv_candidates = [
    'data/processed/reviews_clean.csv',
    'data/processed/reviews_cleaned.csv',
    'data/raw/reviews_raw.csv',
    'data/raw/reviews.csv',
    'data/raw/reviews_clean.csv'
]
if df is None:
    for p in csv_candidates:
        if os.path.exists(p):
            print('Loading CSV fallback:', p)
            df = pd.read_csv(p)
            break

if df is None:
    print('No data found in DB or local CSV candidates.
Please run the scraper/preprocessor or provide DB connection details in environment variables.')
else:
    # Normalize some common column names for downstream analysis
    df = df.rename(columns=lambda s: s.strip() if isinstance(s, str) else s)
    # common fields: review_text or review, rating or score, review_date or date
    if 'review' in df.columns and 'review_text' not in df.columns:
        df = df.rename(columns={'review': 'review_text'})
    if 'score' in df.columns and 'rating' not in df.columns:
        df = df.rename(columns={'score': 'rating'})
    if 'date' in df.columns and 'review_date' not in df.columns:
        df = df.rename(columns={'date': 'review_date'})
    # ensure review_date is datetime where possible
    if 'review_date' in df.columns:
        df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')

    print('
Data loaded. Rows: {}, Columns: {}'.format(len(df), len(df.columns)))

In [ ]:
# Quick look: head / dtypes / missing values
if df is not None:
    display(df.head())
    print(df.dtypes)
    print('
Missing values per column:')
    print(df.isnull().sum())

    # Basic counts
    print('
Total reviews:', len(df))
    if 'rating' in df.columns:
        print('Rating value counts:
', df['rating'].value_counts(dropna=False).sort_index())
    if 'bank_name' in df.columns:
        print('Reviews per bank:
', df['bank_name'].value_counts())
else:
    print('No dataframe loaded')

In [ ]:
# Rating distribution and missing-rate checks
if df is not None and 'rating' in df.columns:
    plt.figure(figsize=(6,4))
    sns.countplot(x='rating', data=df, order=sorted(df['rating'].dropna().unique()))
    plt.title('Rating distribution')
    plt.show()

# Date-based trend (reviews per month)
if df is not None and 'review_date' in df.columns:
    ts = df.dropna(subset=['review_date']).copy()
    ts['year_month'] = ts['review_date'].dt.to_period('M')
    counts = ts.groupby('year_month').size().reset_index(name='n')
    counts['year_month'] = counts['year_month'].dt.to_timestamp()
    plt.figure(figsize=(10,4))
    sns.lineplot(x='year_month', y='n', data=counts, marker='o')
    plt.title('Reviews per month')
    plt.xlabel('Month')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Simple text analysis: word counts and common words
if df is not None and 'review_text' in df.columns:
    import re
    from collections import Counter
    # small stopword list
    STOPWORDS = set(['the','and','a','to','is','in','it','for','of','i','this','that','on','with','was','are','but','be','have','not'])
    words = Counter()
    df['text_length'] = df['review_text'].astype(str).str.len()
    for txt in df['review_text'].dropna().astype(str):
        toks = re.findall(r
, txt.lower())
        toks = [t for t in toks if t not in STOPWORDS and len(t) > 2]
        words.update(toks)
    common = words.most_common(20)
    print('Top 20 words:')
    for w,c in common:
        print(f'{w}: {c}')

    # Histogram of review lengths
    plt.figure(figsize=(6,4))
    sns.histplot(df['text_length'].dropna(), bins=30)
    plt.title('Review text length distribution')
    plt.xlabel('Characters')
    plt.show()

    # Show a few sample reviews
    print('
Sample reviews:')
    display(df[['bank_name','rating','review_date','review_text']].head(10))
else:
    print('No review_text column available for text analysis')

## Next steps
- Validate that the dataset meets the Task 1 KPI: >=400 reviews per bank and <5% missing critical data.
- If DB contains more fields (device, version), extend analysis to see opinion drift by app version.
- Add sentiment analysis and topic modelling in a subsequent notebook cell.